<a href="https://colab.research.google.com/github/lakshh670/100_days_of_DeepLearning/blob/main/HyperParameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#IN this notebook we will perform hyper-parameter tuning on a classification dataset.
# The main goal of hyper-paramter tuning is to find the perfect:
          #Optimizer,loss-function,no. of layers, no. of nodes per layer.
# 1st we will hypertune individually and then multiple at a time

In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
x,y=df.iloc[:,:-1],df.iloc[:,-1]
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x=scaler.fit_transform(x)
x.shape

(768, 8)

In [5]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

# Optimiser

In [9]:
# For this we will use a library called keras tuner
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.0 MB/s eta 0:00:00


In [64]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import kerastuner as kt

In [24]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(32,input_dim=8,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  optimizer=hp.Choice('optimizer',values=['sgd','rmsprop','adam','adadelta'])
  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [25]:
tuner=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
tuner.search(x_train,y_train,validation_data=(x_test,y_test),epochs=5) # Only 5 eopchs are enough to check for the best optimizer

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.6979166865348816

Best val_accuracy So Far: 0.78125
Total elapsed time: 00h 00m 09s


In [28]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [31]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [32]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model.fit(x_train,y_train,batch_size=x.shape[0],validation_data=(x_test,y_test),initial_epoch=6,epochs=100)

Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7431 - loss: 0.5616 - val_accuracy: 0.7812 - val_loss: 0.5462
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.7448 - loss: 0.5592 - val_accuracy: 0.7865 - val_loss: 0.5442
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.7483 - loss: 0.5569 - val_accuracy: 0.7865 - val_loss: 0.5422
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.7465 - loss: 0.5546 - val_accuracy: 0.7865 - val_loss: 0.5404
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.7517 - loss: 0.5523 - val_accuracy: 0.7917 - val_loss: 0.5386
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.7535 - loss: 0.5501 - val_accuracy: 0.7917 - val_loss: 0.5368
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.7604 - loss: 0.5480 - val_accuracy: 0.7969 - val_loss: 0.5351
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.7622 - loss: 0.5458 - val_accuracy: 0.7969 - val_lo

# No. of Nodes

In [11]:
def build_model1(hp):
  model=Sequential()
  units=hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(Dense(units=units,input_dim=8,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [12]:
tuner=kt.RandomSearch(build_model1,
                       objective='val_accuracy',
                        max_trials=5,
                         directory='my_dir')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
tuner.search(x_train,y_train,validation_data=(x_test,y_test),epochs=5)

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 13s


In [14]:
tuner.get_best_hyperparameters()[0].values

{'units': 72}

In [15]:
model=tuner.get_best_models(1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 72)             │           648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            73 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 721 (2.82 KB)

 Trainable params: 721 (2.82 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,initial_epoch=6,batch_size=x.shape[0])

Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7687 - loss: 0.4960 - val_accuracy: 0.8117 - val_loss: 0.4935
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.7752 - loss: 0.4940 - val_accuracy: 0.8117 - val_loss: 0.4915
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.7752 - loss: 0.4920 - val_accuracy: 0.8117 - val_loss: 0.4895
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.7736 - loss: 0.4901 - val_accuracy: 0.8117 - val_loss: 0.4877
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.7752 - loss: 0.4882 - val_accuracy: 0.8117 - val_loss: 0.4858
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.7752 - loss: 0.4864 - val_accuracy: 0.8117 - val_loss: 0.4841
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.7752 - loss: 0.4846 - val_accuracy: 0.8117 - val_loss: 0.4824
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.7752 - loss: 0.4828 - val_accuracy: 0.8182 - val_l

# Num of Layers

In [19]:
def build_model2(hp):
  model=Sequential()
  model.add(Dense(72,activation='relu',input_dim=8))
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(72,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [20]:
tuner=kt.RandomSearch(build_model2
                      ,max_trials=3
                      , objective='val_accuracy'
                      , directory='my_dir'
                      , project_name='num_layers')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 17s


In [23]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [24]:
model=tuner.get_best_models()[0]

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [27]:
model.fit(x_train,y_train,initial_epoch=7,epochs=100,validation_data=(x_test,y_test),batch_size=x.shape[0])

Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.7638 - loss: 0.5636 - val_accuracy: 0.7727 - val_loss: 0.5421
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.7590 - loss: 0.5483 - val_accuracy: 0.7532 - val_loss: 0.5314
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.7573 - loss: 0.5352 - val_accuracy: 0.7597 - val_loss: 0.5198
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.7622 - loss: 0.5224 - val_accuracy: 0.7727 - val_loss: 0.5076
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.7704 - loss: 0.5095 - val_accuracy: 0.7987 - val_loss: 0.4955
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.7752 - loss: 0.4974 - val_accuracy: 0.8052 - val_loss: 0.4848
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.7850 - loss: 0.4865 - val_accuracy: 0.8182 - val_loss: 0.4763
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.7899 - loss: 0.4771 - val_accuracy: 0.8052 - val

# Now we will combine all the things together

In [65]:
# We are also adding DropOut after every layer
def build_model3(hp):
  model=Sequential()
  counter=0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if counter==0:
      model.add(Dense(hp.Int(f'nodes + {i}layers',min_value=8,max_value=256,step=8),input_dim=8,activation=hp.Choice(f'activation + {i} layers',values=['relu','sigmoid','tanh'])))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(Dense(hp.Int(f'nodes + {i}layers',min_value=8,max_value=256,step=8),activation=hp.Choice(f'activation + {i} layers',values=['relu','sigmoid','tanh'])))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice(f'optimizer',values=['adam','rmsprop','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [66]:
tuner=kt.RandomSearch(build_model3
                      ,max_trials=3
                      , objective='val_accuracy'
                      , directory='my_dir'
                      , project_name='final3')

In [67]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 14s


In [61]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3,
 'nodes + 0layers': 80,
 'activation + 0 layers': 'relu',
 'optimizer': 'rmsprop',
 'nodes + 1layers': 248,
 'activation + 1 layers': 'tanh',
 'nodes + 2layers': 160,
 'activation + 2 layers': 'tanh',
 'nodes + 3layers': 168,
 'activation + 3 layers': 'sigmoid',
 'nodes + 4layers': 216,
 'activation + 4 layers': 'tanh',
 'nodes + 5layers': 160,
 'activation + 5 layers': 'relu',
 'nodes + 6layers': 232,
 'activation + 6 layers': 'sigmoid',
 'nodes + 7layers': 152,
 'activation + 7 layers': 'relu',
 'nodes + 8layers': 16,
 'activation + 8 layers': 'relu'}

In [68]:
model=tuner.get_best_models()[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [69]:
model.fit(x_train,y_train,initial_epoch=7,epochs=100,validation_data=(x_test,y_test),batch_size=x.shape[0])

Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7248 - loss: 0.5077 - val_accuracy: 0.7987 - val_loss: 0.4869
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.7492 - loss: 0.4918 - val_accuracy: 0.8117 - val_loss: 0.4859
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.7443 - loss: 0.4996 - val_accuracy: 0.8052 - val_loss: 0.4849
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.7394 - loss: 0.4954 - val_accuracy: 0.8052 - val_loss: 0.4840
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.7378 - loss: 0.4917 - val_accuracy: 0.8052 - val_loss: 0.4833
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.7313 - loss: 0.4979 - val_accuracy: 0.8052 - val_loss: 0.4826
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.7329 - loss: 0.5109 - val_accuracy: 0.8052 - val_loss: 0.4820
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.7443 - loss: 0.5067 - val_accuracy: 0.8052 - va